# Обучение Word2Vec на своих данных

### Загружаем и готовим данные отзывов на банки

In [1]:
!pip install pymorphy2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 45.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=777d2d6b6d6e0926d3623a8fbdc37e8fd3c50cd44ec0516ed5274ebb0bb3c3e9
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt


In [2]:
import pandas as pd
import pymorphy2
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

In [3]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [4]:
def preprocess(text, stop_words, punctuation_marks, morph):
    tokens = word_tokenize(text.lower())
    preprocessed_text = []
    for token in tokens:
        if token not in punctuation_marks:
            lemma = morph.parse(token)[0].normal_form
            if lemma not in stop_words:
                preprocessed_text.append(lemma)
    return preprocessed_text

In [5]:
punctuation_marks = ['!', ',', '(', ')', ':', '-', '?', '.', '..', '...']
stop_words = stopwords.words("russian")
morph = pymorphy2.MorphAnalyzer()

In [6]:
!wget https://huggingface.co/datasets/C0uchP0tat0/courses_obrazoval_ru/resolve/main/reviews.csv -O dataset.csv

--2024-04-27 19:30:47--  https://huggingface.co/datasets/C0uchP0tat0/courses_obrazoval_ru/resolve/main/reviews.csv
Resolving huggingface.co (huggingface.co)... 18.154.227.69, 18.154.227.7, 18.154.227.67, ...
Connecting to huggingface.co (huggingface.co)|18.154.227.69|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.huggingface.co/repos/ea/0f/ea0f960c15d93f8470907f7f90de2432df5691b359e631cdc5619075be9ca37b/591525c1812ffd8c66d8c39628e923bbe818fb9f4a0073c8c6f93d98ede84432?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27reviews.csv%3B+filename%3D%22reviews.csv%22%3B&response-content-type=text%2Fcsv&Expires=1714505447&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcxNDUwNTQ0N319LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmh1Z2dpbmdmYWNlLmNvL3JlcG9zL2VhLzBmL2VhMGY5NjBjMTVkOTNmODQ3MDkwN2Y3ZjkwZGUyNDMyZGY1NjkxYjM1OWU2MzFjZGM1NjE5MDc1YmU5Y2EzN2IvNTkxNTI1YzE4MTJmZmQ4YzY2ZDhjMzk2MjhlOTI

In [7]:
courses = pd.read_csv('dataset.csv', sep=',', index_col=False)

In [8]:
courses

,content,label
0,"Сложная тема, которая стала понятна благодаря ...",positive
1,Большое спасибо команде Nolim за создание тако...,positive
2,Отличный курс! Не многие умеют делать кастомны...,positive
3,"Курс, который превзошёл мои ожидания и даже бо...",positive
4,Закончила уже не один курс. Шла с абсолютного ...,positive
...,...,...
31385,Ужасное отношение менеджеров по телефону к пот...,negative
31386,Я занимаюсь второй год и для меня это самый уд...,negative
31387,"Skyeng очень крутая школа, она отличается от м...",negative
31388,Чудесная школа. Второй преподаватель и второй ...,negative


In [9]:
courses['Preprocessed_texts'] = courses.apply(lambda row: preprocess(row['content'], punctuation_marks, stop_words, morph), axis=1)

In [10]:
courses

,content,label,Preprocessed_texts
0,"Сложная тема, которая стала понятна благодаря ...",positive,"[сложный, тема, который, стать, понятный, благ..."
1,Большое спасибо команде Nolim за создание тако...,positive,"[большой, спасибо, команда, nolim, создание, т..."
2,Отличный курс! Не многие умеют делать кастомны...,positive,"[отличный, курс, многие, уметь, делать, кастом..."
3,"Курс, который превзошёл мои ожидания и даже бо...",positive,"[курс, который, превзойти, мой, ожидание, ребя..."
4,Закончила уже не один курс. Шла с абсолютного ...,positive,"[закончить, курс, идти, абсолютный, нуль, обла..."
...,...,...,...
31385,Ужасное отношение менеджеров по телефону к пот...,negative,"[ужасный, отношение, менеджер, телефон, потенц..."
31386,Я занимаюсь второй год и для меня это самый уд...,negative,"[заниматься, второй, год, это, самый, удобный,..."
31387,"Skyeng очень крутая школа, она отличается от м...",negative,"[skyeng, очень, крутой, школа, отличаться, мно..."
31388,Чудесная школа. Второй преподаватель и второй ...,negative,"[чудесный, школа, второй, преподаватель, второ..."


## Обучение Word2Vec на отзывах на курсы

In [11]:
import gensim.models

In [12]:
model = gensim.models.Word2Vec(sentences=courses['Preprocessed_texts'],
                               min_count=5,
                               vector_size=100)

In [13]:
model.wv['бухгалтер']

array([-0.1068092 ,  0.43516594,  0.22456184, -0.1448835 ,  0.50031376,
       -0.32486722, -0.07584585,  0.25191912, -0.06769393,  0.03031802,
        0.10965192, -0.24386694,  0.22641489,  0.3202905 , -0.05831206,
       -0.5819721 , -0.11410866, -0.0934022 , -0.04406779, -0.22800082,
       -0.13568547, -0.11089952,  0.13271685, -0.1107417 ,  0.16987005,
        0.09588698, -0.06813823, -0.27595732, -0.07956672,  0.01570301,
        0.0903529 ,  0.10152806,  0.3621105 ,  0.11410727,  0.03913584,
        0.52717716,  0.23088843,  0.130022  ,  0.12786895, -0.65475327,
       -0.32598242, -0.6027341 ,  0.0929001 ,  0.07414787, -0.07540827,
       -0.27703825, -0.27679113, -0.08091893, -0.03641225, -0.21825616,
        0.29957587,  0.23866275, -0.06806278,  0.08208349,  0.19274397,
       -0.11541066,  0.16368103, -0.44594786, -0.5584594 , -0.20646536,
        0.3109425 , -0.12207062, -0.2087742 , -0.20478587, -0.03014391,
       -0.0788869 , -0.05587458, -0.22053961, -0.45700365, -0.11

In [14]:
model.wv['работать']

array([ 1.3236282 ,  0.6504674 , -1.1811246 ,  1.8146334 , -1.3030102 ,
        0.27527782, -0.11089668,  1.8942302 , -0.861187  , -0.05094508,
        0.21561971, -0.1290091 , -0.04897958,  1.3747808 , -0.17828146,
       -1.5884733 , -2.039796  ,  0.1699948 ,  0.20585622,  0.5460651 ,
        0.2527037 ,  0.4081952 , -2.1775975 ,  0.5470755 ,  1.0289599 ,
        1.841934  ,  0.8727012 ,  0.0427471 ,  1.0667896 ,  0.12015542,
       -0.0652807 ,  2.0621517 , -0.3430322 , -1.1329514 , -2.1900232 ,
        2.7630274 ,  0.3018068 ,  1.7018721 ,  0.58817345, -0.6910452 ,
       -0.8004553 , -0.77478206, -1.2566233 , -2.2622027 , -1.5155665 ,
       -2.5973134 , -0.01667626,  0.39545313,  1.4011222 , -0.07607265,
       -1.5207853 ,  1.375021  ,  0.28179765, -0.6276102 ,  0.06797972,
        1.4745853 ,  0.4787067 ,  0.02698116, -0.5485599 , -1.0048791 ,
        1.3683889 ,  1.8143747 , -0.13687102, -1.7297412 ,  0.0467583 ,
        0.31717998,  0.07327303, -1.1502973 ,  0.887603  ,  0.31

In [15]:
model.wv['замечательный']

array([ 0.6747833 ,  0.62225515, -2.072009  ,  1.1633898 , -0.73547286,
        0.29422262,  2.0411193 ,  0.12661685, -0.7646168 , -0.29310718,
        0.04224992,  0.15086909,  0.15946071, -1.0573983 , -1.6514362 ,
       -0.40354759,  1.3076105 , -0.1839873 , -0.4036643 ,  0.525284  ,
       -0.06915253,  0.36315244, -0.35772714, -0.108487  ,  0.7892658 ,
       -0.06218191, -1.4627811 ,  0.9230988 , -0.34478244,  1.0085263 ,
       -0.6622544 ,  0.17325824, -0.6820403 , -0.04014499, -0.6592244 ,
       -0.16313432, -0.37581098,  0.9600906 ,  1.4904304 ,  0.3567813 ,
        0.95417166,  1.3561181 ,  0.28001466, -0.7515888 , -0.59453547,
        0.6826957 , -0.07319835, -0.29830387,  0.56436807,  1.1564156 ,
       -1.014847  , -0.77586395,  0.13168754,  1.131304  ,  0.6966366 ,
        0.46547544, -0.893402  ,  0.09428343,  1.5473346 ,  0.2218182 ,
        0.33280337, -0.84478337,  0.26566055, -2.3676326 , -0.81669843,
        0.10515035,  0.38145503, -0.03648059, -0.8576177 , -1.03

In [16]:
model.wv.most_similar('замечательный')

[('прекрасный', 0.8660831451416016),
 ('чудесный', 0.8212152719497681),
 ('потрясать', 0.7714234590530396),
 ('компетентный', 0.7595062255859375),
 ('молодой', 0.7361327409744263),
 ('великолепный', 0.732946515083313),
 ('шикарный', 0.7306846976280212),
 ('отличный', 0.7261954545974731),
 ('грамотный', 0.6951461434364319),
 ('классный', 0.6758410334587097)]

In [17]:
model.wv.most_similar('стрессовый')

[('прогрессия', 0.7302366495132446),
 ('филигранный', 0.729767918586731),
 ('подмечать', 0.7261526584625244),
 ('огонёк', 0.7229665517807007),
 ('тонкий', 0.7184333205223083),
 ('ускорять', 0.715752124786377),
 ('расстройство', 0.7099881768226624),
 ('прокрастинация', 0.7094666957855225),
 ('пунктуационный', 0.7039498686790466),
 ('раскадровка', 0.7015565633773804)]

## Сохранение обученной модели

In [18]:
model.save('word2vec-obrazoval.ru-50')

In [19]:
!ls -lh

total 53M
-rw-r--r-- 1 root root  41M Apr 25 07:56 dataset.csv
drwxr-xr-x 1 root root 4.0K Apr 25 13:25 sample_data
-rw-r--r-- 1 root root  49K Apr 27 19:30 train_data.csv
-rw-r--r-- 1 root root  12M Apr 27 19:41 word2vec-obrazoval.ru-50


## Загрузка сохраненной модели

In [20]:
new_model = gensim.models.Word2Vec.load('word2vec-obrazoval.ru-50')

In [21]:
new_model.wv['работать']

array([ 1.3236282 ,  0.6504674 , -1.1811246 ,  1.8146334 , -1.3030102 ,
        0.27527782, -0.11089668,  1.8942302 , -0.861187  , -0.05094508,
        0.21561971, -0.1290091 , -0.04897958,  1.3747808 , -0.17828146,
       -1.5884733 , -2.039796  ,  0.1699948 ,  0.20585622,  0.5460651 ,
        0.2527037 ,  0.4081952 , -2.1775975 ,  0.5470755 ,  1.0289599 ,
        1.841934  ,  0.8727012 ,  0.0427471 ,  1.0667896 ,  0.12015542,
       -0.0652807 ,  2.0621517 , -0.3430322 , -1.1329514 , -2.1900232 ,
        2.7630274 ,  0.3018068 ,  1.7018721 ,  0.58817345, -0.6910452 ,
       -0.8004553 , -0.77478206, -1.2566233 , -2.2622027 , -1.5155665 ,
       -2.5973134 , -0.01667626,  0.39545313,  1.4011222 , -0.07607265,
       -1.5207853 ,  1.375021  ,  0.28179765, -0.6276102 ,  0.06797972,
        1.4745853 ,  0.4787067 ,  0.02698116, -0.5485599 , -1.0048791 ,
        1.3683889 ,  1.8143747 , -0.13687102, -1.7297412 ,  0.0467583 ,
        0.31717998,  0.07327303, -1.1502973 ,  0.887603  ,  0.31

## Полезные ссылки

1. [Библиотека Gensim](https://radimrehurek.com/gensim/index.html).
2. [Word2Vec в Gensim](https://radimrehurek.com/gensim/auto_examples/tutorials/run_word2vec.html).


# Логистическая регрессия

In [ ]:
!wget https://huggingface.co/datasets/C0uchP0tat0/courses_sravni_ru/resolve/main/dataset.csv -O reviews.csv

--2024-04-26 06:48:29--  https://huggingface.co/datasets/C0uchP0tat0/courses_sravni_ru/resolve/main/dataset.csv
Resolving huggingface.co (huggingface.co)... 65.8.243.16, 65.8.243.90, 65.8.243.92, ...
Connecting to huggingface.co (huggingface.co)|65.8.243.16|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.huggingface.co/repos/e9/a2/e9a241e4099b8361fb6be16dd66c1146a8b8d9805e7fcfef45c7e71d3aee5d80/4d78714bc987232e514f3c87fd37a71fc0187a5d303b13f5a19ee3c9326031f5?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27dataset.csv%3B+filename%3D%22dataset.csv%22%3B&response-content-type=text%2Fcsv&Expires=1714373309&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcxNDM3MzMwOX19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmh1Z2dpbmdmYWNlLmNvL3JlcG9zL2U5L2EyL2U5YTI0MWU0MDk5YjgzNjFmYjZiZTE2ZGQ2NmMxMTQ2YThiOGQ5ODA1ZTdmY2ZlZjQ1YzdlNzFkM2FlZTVkODAvNGQ3ODcxNGJjOTg3MjMyZTUxNGYzYzg3ZmQzN2E3MWZjMDE4N

In [ ]:
# !wget https://huggingface.co/datasets/C0uchP0tat0/courses_obrazoval_ru/resolve/main/reviews.csv -O reviews.csv

In [ ]:
import pandas as pd
import numpy as np
from gensim.models import Word2Vec
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import FunctionTransformer
import pymorphy2
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

nltk.download('punkt')
nltk.download('stopwords')

# Загрузка предварительно обученной модели word2vec
model = Word2Vec.load("word2vec-obrazoval.ru-50")

# Загрузка данных
data = pd.read_csv("reviews.csv")

# Предобработка текста
def preprocess(text, stop_words, punctuation_marks, morph):
    tokens = word_tokenize(text.lower())
    preprocessed_text = []
    for token in tokens:
        if token not in punctuation_marks:
            lemma = morph.parse(token)[0].normal_form
            if lemma not in stop_words:
                preprocessed_text.append(lemma)
    return " ".join(preprocessed_text)

punctuation_marks = ['!', ',', '(', ')', ':', '-', '?', '.', '..', '...']
stop_words = stopwords.words("russian")
morph = pymorphy2.MorphAnalyzer()

try:
    # для датасета СРАВНИ РУ https://huggingface.co/datasets/C0uchP0tat0/courses_sravni_ru
    data['content'] = data['text'].apply(lambda x: preprocess(x, stop_words, punctuation_marks, morph))
except KeyError:
    # для датасета ОБРАЗОВАЛ РУ https://huggingface.co/datasets/C0uchP0tat0/courses_obrazoval_ru
    data['content'] = data['content'].apply(lambda x: preprocess(x, stop_words, punctuation_marks, morph))

# Разделение на признаки и метки
X = data['content']
y = data['label']

# Разделение на обучающий и тестовый наборы
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Функция для преобразования текста в векторы с использованием word2vec
def word_averaging(words, model):
    all_words, mean = set(), []

    for word in words:
        if word in model.wv.key_to_index:
            mean.append(model.wv.get_vector(word))
            all_words.add(model.wv.key_to_index[word])

    if not mean:
        return np.zeros(model.vector_size,)

    mean = np.array(mean).mean(axis=0)
    return mean

def word_vectorizer(X, model):
    return np.array([word_averaging(words.split(), model) for words in X])

# Создание pipeline
w2v_pipeline = Pipeline([
    ("word_vectorizer", FunctionTransformer(word_vectorizer, kw_args={"model": model})),
    ("clf", LogisticRegression())
])

# Обучение модели и оценка ее точности
w2v_pipeline.fit(X_train, y_train)
y_pred = w2v_pipeline.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Accuracy: 0.9398879266428936


In [ ]:
text = """Брал кредит в Мегабанке на автомобиль. Выдали за один день. Никаких скрытых комиссий и переплат.
У банка удобное мобильное приложение, через которое можно быстро отправить ежемесячный платеж.
Досрочное гасить начал через три месяца. Я доволен оперативностью и удобством. Огромное спасибо!
"""
preprocessed_text = preprocess(text, stop_words, punctuation_marks, morph)
predicted_label = w2v_pipeline.predict([preprocessed_text])[0]
print("Predicted label:", predicted_label)

Predicted label: positive


In [ ]:
text = """Взял кредит в ТакСебеБанке на автомобиль. В договор включили обязательный контракт
на помощь на дороге, который мне не нужен. Узнал об этом только во время подписания договора, иначе бы отказался.
Альтернативы была страхование жизни, но мне это даже не предложили. Скорее всего, менеджер продвигает
продажи услуг этой компании в ущерб интересов клиента. Как минимум, непорядочно и непрофессионально.
У банка ужасное мобильное приложение, из-за которого с меня взяли штраф 10 тыс.руб. По требованиям
банка после покупки автомобиля в приложении нужно загрузить ПТС. Я загрузил и проверил, что ПТС в приложении есть.
Но через некоторое время ПТС из приложения пропал и с меня взяли штраф. Никому не рекомендую связываться с ТакСебеБанком.
"""
preprocessed_text = preprocess(text, stop_words, punctuation_marks, morph)
predicted_label = w2v_pipeline.predict([preprocessed_text])[0]
print("Predicted label:", predicted_label)

Predicted label: negative


In [ ]:
mappingId2Label = {0:'negative', 1:'positive'}

In [ ]:
test_data = pd.DataFrame({
    "text": [
        "Этот курс был удивительным! Я много узнал и преподаватель был очень компетентным.",
        "Содержание курса было несвязанным и плохо объяснено.",
        "Я ничего не понял в этом курсе. Потеря времени и денег.",
        "Материал курса был хорошо структурирован и легко воспринимался.",
        "Преподаватель казался неинтересным и не предоставлял много руководства.",
        "Отличный курс! Очень рекомендую его всем, кто хочет учиться.",
        "Курс был запутанным и не соответствовал моим ожиданиям.",
        "Я нашел этот курс очень информативным и полезным для своей карьеры.",
        "Курс был полным временем. Я ничего нового не узнал.",
        "Преподаватель был заинтересованным и сделал материал легким для понимания.",
        "Каки каки каки каки"
    ],
    "label": [1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0]
})

In [ ]:
for r in zip(test_data["text"], test_data["label"]):
    preprocessed_text = preprocess(r[0], stop_words, punctuation_marks, morph)
    predicted_label = w2v_pipeline.predict([preprocessed_text])[0]
    print("Predicted label:", predicted_label, mappingId2Label[r[1]])

Predicted label: positive positive
Predicted label: negative negative
Predicted label: negative negative
Predicted label: positive positive
Predicted label: positive negative
Predicted label: positive positive
Predicted label: negative negative
Predicted label: positive positive
Predicted label: positive negative
Predicted label: positive positive
Predicted label: negative negative


In [ ]:
from sklearn.metrics import f1_score

# Предобработка текста
test_data['content'] = test_data['text'].apply(lambda x: preprocess(x, stop_words, punctuation_marks, morph))

# Предсказание меток для тестовых данных
y_true = test_data['label']
y_pred = w2v_pipeline.predict(test_data['content'])

# Преобразование предсказанных меток в числовой формат
y_pred = [1 if label == 'positive' else 0 for label in y_pred]

# Оценка F1-меры
f1 = f1_score(y_true, y_pred)
print("F1-мера на тестовых данных:", f1)

F1-мера на тестовых данных: 0.8333333333333333


In [ ]:
# Предсказание вероятностей классов для тестовых данных
probabilities = w2v_pipeline.predict_proba(X_test)

# Отсортировать индексы отзывов по вероятностям позитивного класса
sorted_positive_indices = np.argsort(probabilities[:, 1])[::-1]

# Отсортировать индексы отзывов по вероятностям негативного класса
sorted_negative_indices = np.argsort(probabilities[:, 0])[::-1]

most_positive_reviews = X_test.iloc[sorted_positive_indices[:50]]
most_negative_reviews = X_test.iloc[sorted_negative_indices[:50]]

most_positive = []
most_negative = []

# Вывести 50 самых позитивных отзывов
print("Самые позитивные отзывы:")
for i, review in enumerate(most_positive_reviews):
    most_positive.append(data["text"][most_positive_reviews.index[i]])
    print(f"Отзыв #{i+1}, индекс строки: {most_positive_reviews.index[i]}")
    print(review)
    print()

# Вывести 50 самых негативных отзывов
print("Самые негативные отзывы:")
for i, review in enumerate(most_negative_reviews):
    most_negative.append(data["text"][most_negative_reviews.index[i]])
    print(f"Отзыв #{i+1}, индекс строки: {most_negative_reviews.index[i]}")
    print(review)
    print()

In [ ]:
import gensim
from gensim import corpora

# Создание словаря
dictionary = corpora.Dictionary(most_positive_reviews.apply(lambda x: x.split()))

# Преобразование текста в мешок слов
corpus = [dictionary.doc2bow(text.split()) for text in most_positive_reviews]

# Обучение модели LDA
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                            id2word=dictionary,
                                            num_topics=5,
                                            random_state=42,
                                            passes=10,
                                            per_word_topics=True)

# Вывод тем
print(lda_model.print_topics())

NameError: name 'most_positive_reviews' is not defined

In [ ]:
import gensim
from gensim import corpora

# Создание словаря
dictionary = corpora.Dictionary(most_negative_reviews.apply(lambda x: x.split()))

# Преобразование текста в мешок слов
corpus = [dictionary.doc2bow(text.split()) for text in most_negative_reviews]

# Обучение модели LDA
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                            id2word=dictionary,
                                            num_topics=5,
                                            random_state=42,
                                            passes=10,
                                            per_word_topics=True)

# Вывод тем
print(lda_model.print_topics())

[(0, '0.020*"деньга" + 0.017*"обучение" + 0.016*"вернуть" + 0.012*"возврат" + 0.010*"курс" + 0.008*"это" + 0.008*"договор" + 0.007*"день" + 0.007*"свой" + 0.006*"заявление"'), (1, '0.015*"возврат" + 0.015*"день" + 0.012*"это" + 0.012*"деньга" + 0.011*"обучение" + 0.008*"вернуть" + 0.008*"менеджер" + 0.008*"курс" + 0.008*"заявление" + 0.007*"решить"'), (2, '0.010*"возврат" + 0.009*"деньга" + 0.008*"средство" + 0.007*"обучение" + 0.007*"договор" + 0.007*"день" + 0.007*"30" + 0.006*"месяц" + 0.006*"сказать" + 0.006*"копия"'), (3, '0.020*"это" + 0.015*"обучение" + 0.012*"который" + 0.011*">" + 0.011*"<" + 0.010*"деньга" + 0.007*"курс" + 0.007*"вернуть" + 0.006*"мочь" + 0.006*"урок"'), (4, '0.017*"курс" + 0.015*"деньга" + 0.011*"вернуть" + 0.011*"обучение" + 0.009*"это" + 0.009*"день" + 0.008*"который" + 0.007*"\'\'" + 0.007*"написать" + 0.007*"возврат"')]


In [ ]:
from sklearn.decomposition import NMF
from sklearn.feature_extraction.text import TfidfVectorizer

# Преобразование текста в TF-IDF признаки
tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2, stop_words='english')
tfidf = tfidf_vectorizer.fit_transform(most_positive_reviews)

# Обучение модели NMF
num_topics = 10
nmf_model = NMF(n_components=num_topics, random_state=42)
nmf_model.fit(tfidf)

# Вывод тем
feature_names = tfidf_vectorizer.get_feature_names_out()
for topic_idx, topic in enumerate(nmf_model.components_):
    print(f"Тема {topic_idx}:")
    print(" ".join([feature_names[i] for i in topic.argsort()[:-10 - 1:-1]]))
    print()

Тема 0:
ребёнок нравиться заниматься очень интерес удовольствие форма урок отличный педагог

Тема 1:
обучение удобный формат большой онлайн эффективный программа выбор подготовка проходить

Тема 2:
английский год платформа результат язык говорение говорить очень заниматься изучение

Тема 3:
очень советовать школа хороший весь самый эффективно интересный урок несколько

Тема 4:
дочка нравиться тетрик индивидуальный подход математика заниматься спасибо наш огромный

Тема 5:
рекомендовать всё школа довольный ребёнок понятно доступно объяснять делать очень

Тема 6:
русский математика язык заниматься предмет ребёнок всё подтянуть занятие хороший

Тема 7:
разный курс разработка профессиональный отзывчивый направление свой большой знание хотеть

Тема 8:
интересно задание занятие заниматься выбрать время удобно skyeng преподаватель приложение

Тема 9:
работа подача спасибо материал найти обучение проходить удобно очень нужно



In [ ]:
from sklearn.decomposition import NMF
from sklearn.feature_extraction.text import TfidfVectorizer

# Преобразование текста в TF-IDF признаки
tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2, stop_words='english')
tfidf = tfidf_vectorizer.fit_transform(most_negative_reviews)

# Обучение модели NMF
num_topics = 10
nmf_model = NMF(n_components=num_topics, random_state=42)
nmf_model.fit(tfidf)

# Вывод тем
feature_names = tfidf_vectorizer.get_feature_names_out()
for topic_idx, topic in enumerate(nmf_model.components_):
    print(f"Тема {topic_idx}:")
    print(" ".join([feature_names[i] for i in topic.argsort()[:-10 - 1:-1]]))
    print()

In [ ]:
!pip install bigartm

In [ ]:
import artm
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer

# Загрузка данных
data = pd.read_csv("reviews.csv")

# Предварительная обработка текста
nltk.download('stopwords')
nltk.download('punkt')
stop_words = set(stopwords.words('russian'))

def preprocess_text(text):
    tokens = word_tokenize(text.lower())
    lemmatized_tokens = [token for token in tokens if token.isalpha() and token not in stop_words]
    return " ".join(lemmatized_tokens)

data['processed_content'] = data['content'].apply(preprocess_text)

# Разделение на обучающий и тестовый наборы
train_data, test_data = train_test_split(most_positive_reviews, test_size=0.2, random_state=42)

# Создание и обучение CountVectorizer
cv = CountVectorizer(max_features=1000)
n_wd = cv.fit_transform(train_data).toarray().T
vocabulary = cv.get_feature_names_out()

# Создание BatchVectorizer
bv = artm.BatchVectorizer(data_format='bow_n_wd', n_wd=n_wd, vocabulary=vocabulary)

# Обучение модели LDA
model = artm.LDA(num_topics=15, dictionary=bv.dictionary)
model.fit_offline(bv, num_collection_passes=20)

# Вывод результатов
print(model.get_top_tokens())


In [ ]:
import artm
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer

# Разделение на обучающий и тестовый наборы
train_data, test_data = train_test_split(most_negative_reviews, test_size=0.2, random_state=42)

# Создание и обучение CountVectorizer
cv = CountVectorizer(max_features=1000)
n_wd = cv.fit_transform(train_data).toarray().T
vocabulary = cv.get_feature_names_out()

# Создание BatchVectorizer
bv = artm.BatchVectorizer(data_format='bow_n_wd', n_wd=n_wd, vocabulary=vocabulary)

# Обучение модели LDA
model = artm.LDA(num_topics=15, dictionary=bv.dictionary)
model.fit_offline(bv, num_collection_passes=20)

# Вывод результатов
print(model.get_top_tokens())


In [ ]:
import nltk
from nltk.collocations import *
import pandas as pd

def calculate_bigram_rankings(tokenized_texts, N_best=100):
    texts = [t for sublist in tokenized_texts for t in sublist]
    bigram_measures = nltk.collocations.BigramAssocMeasures()
    finder = BigramCollocationFinder.from_words(texts)
    finder.apply_freq_filter(1)

    raw_freq_ranking = [' '.join(i) for i in finder.nbest(bigram_measures.raw_freq, N_best)]
    tscore_ranking = [' '.join(i) for i in finder.nbest(bigram_measures.student_t, N_best)]
    pmi_ranking =  [' '.join(i) for i in finder.nbest(bigram_measures.pmi, N_best)]
    llr_ranking = [' '. join(i) for i in finder.nbest(bigram_measures.likelihood_ratio, N_best)]
    chi2_ranking =  [' '.join(i) for i in finder.nbest(bigram_measures.chi_sq, N_best)]

    rankings = pd.DataFrame({
        'raw_freq': raw_freq_ranking,
        'pmi': pmi_ranking,
        't-score': tscore_ranking,
        'chi2': chi2_ranking,
        'llr': llr_ranking
    })
    rankings = rankings[['raw_freq', 'pmi', 't-score', 'chi2', 'llr']]
    return rankings

def print_bigram_ranking(ranking):
    for measure, bigrams in ranking.items():
        print(f"Топ-100 биграм по мере {measure}:")
        for i, bigram in enumerate(bigrams, 1):
            print(f"{i}. {bigram}")
        print()

# Пример использования
# tokenized_texts = [text.split() for text in most_positive_reviews]
tokenized_texts = [text.split() for text in most_negative_reviews]
rankings = calculate_bigram_rankings(tokenized_texts)
print(rankings.head(10))
print()

# Вывод ранжированных биграм
print_bigram_ranking(rankings)


NameError: name 'most_negative_reviews' is not defined

# Стохастический градиентный спуск с логистической функцией потерь

In [22]:
!wget https://huggingface.co/datasets/C0uchP0tat0/courses_sravni_ru/resolve/main/dataset.csv -O reviews.csv

--2024-04-27 19:41:09--  https://huggingface.co/datasets/C0uchP0tat0/courses_sravni_ru/resolve/main/dataset.csv
Resolving huggingface.co (huggingface.co)... 18.154.227.67, 18.154.227.7, 18.154.227.87, ...
Connecting to huggingface.co (huggingface.co)|18.154.227.67|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.huggingface.co/repos/e9/a2/e9a241e4099b8361fb6be16dd66c1146a8b8d9805e7fcfef45c7e71d3aee5d80/4d78714bc987232e514f3c87fd37a71fc0187a5d303b13f5a19ee3c9326031f5?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27dataset.csv%3B+filename%3D%22dataset.csv%22%3B&response-content-type=text%2Fcsv&Expires=1714506069&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcxNDUwNjA2OX19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmh1Z2dpbmdmYWNlLmNvL3JlcG9zL2U5L2EyL2U5YTI0MWU0MDk5YjgzNjFmYjZiZTE2ZGQ2NmMxMTQ2YThiOGQ5ODA1ZTdmY2ZlZjQ1YzdlNzFkM2FlZTVkODAvNGQ3ODcxNGJjOTg3MjMyZTUxNGYzYzg3ZmQzN2E3MW

In [23]:
# !wget https://huggingface.co/datasets/C0uchP0tat0/courses_obrazoval_ru/resolve/main/reviews.csv -O reviews.csv

In [24]:
import pandas as pd
import numpy as np
from gensim.models import Word2Vec
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import FunctionTransformer
import pymorphy2
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

# Загрузка предварительно обученной модели word2vec
model = Word2Vec.load("word2vec-obrazoval.ru-50")

# Загрузка данных
data = pd.read_csv("reviews.csv")

# Предобработка текста
def preprocess(text, stop_words, punctuation_marks, morph):
    tokens = word_tokenize(text.lower())
    preprocessed_text = []
    for token in tokens:
        if token not in punctuation_marks:
            lemma = morph.parse(token)[0].normal_form
            if lemma not in stop_words:
                preprocessed_text.append(lemma)
    return " ".join(preprocessed_text)

punctuation_marks = ['!', ',', '(', ')', ':', '-', '?', '.', '..', '...']
stop_words = stopwords.words("russian")
morph = pymorphy2.MorphAnalyzer()

try:
    # для датасета СРАВНИ РУ https://huggingface.co/datasets/C0uchP0tat0/courses_sravni_ru
    data['content'] = data['text'].apply(lambda x: preprocess(x, stop_words, punctuation_marks, morph))
except KeyError:
    # для датасета ОБРАЗОВАЛ РУ https://huggingface.co/datasets/C0uchP0tat0/courses_obrazoval_ru
    data['content'] = data['content'].apply(lambda x: preprocess(x, stop_words, punctuation_marks, morph))

# Разделение на признаки и метки
X = data['content']
y = data['label']

# Разделение на обучающий и тестовый наборы
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Функция для преобразования текста в векторы с использованием word2vec
def word_averaging(words, model):
    all_words, mean = set(), []

    for word in words:
        if word in model.wv.key_to_index:
            mean.append(model.wv.get_vector(word))
            all_words.add(model.wv.key_to_index[word])

    if not mean:
        return np.zeros(model.vector_size,)

    mean = np.array(mean).mean(axis=0)
    return mean

def word_vectorizer(X, model):
    return np.array([word_averaging(words.split(), model) for words in X])

# Создание pipeline
w2v_pipeline = Pipeline([
    ("word_vectorizer", FunctionTransformer(word_vectorizer, kw_args={"model": model})),
    ("clf", SGDClassifier(loss='log', alpha=0.0001, penalty='l2', max_iter=1000, tol=1e-3))
])

# Обучение модели и оценка ее точности
w2v_pipeline.fit(X_train, y_train)
y_pred = w2v_pipeline.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:163: FutureWarning: The loss 'log' was deprecated in v1.1 and will be removed in version 1.3. Use `loss='log_loss'` which is equivalent.
  warnings.warn(


Accuracy: 0.9388690779419256


In [25]:
import joblib

# Сохранение модели
joblib.dump(w2v_pipeline, 'word2vec_pipeline.pkl')

['word2vec_pipeline.pkl']

In [26]:
import joblib

# Загрузка модели
loaded_model = joblib.load('word2vec_pipeline.pkl')

# Предположим, у вас есть данные X_test
# Вы должны загрузить X_test перед использованием модели для предсказания

# Применение загруженной модели
y_pred = loaded_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


Accuracy: 0.9388690779419256


In [27]:
mappingId2Label = {0:'negative', 1:'positive'}

In [28]:
test_data = pd.DataFrame({
    "text": [
        "Этот курс был удивительным! Я много узнал и преподаватель был очень компетентным.",
        "Содержание курса было несвязанным и плохо объяснено.",
        "Я ничего не понял в этом курсе. Потеря времени и денег.",
        "Материал курса был хорошо структурирован и легко воспринимался.",
        "Преподаватель казался неинтересным и не предоставлял много руководства.",
        "Отличный курс! Очень рекомендую его всем, кто хочет учиться.",
        "Курс был запутанным и не соответствовал моим ожиданиям.",
        "Я нашел этот курс очень информативным и полезным для своей карьеры.",
        "Курс был полным временем. Я ничего нового не узнал.",
        "Преподаватель был заинтересованным и сделал материал легким для понимания.",
        "Каки каки каки каки"
    ],
    "label": [1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0]
})

In [29]:
for r in zip(test_data["text"], test_data["label"]):
    preprocessed_text = preprocess(r[0], stop_words, punctuation_marks, morph)
    predicted_label = w2v_pipeline.predict([preprocessed_text])[0]
    print("Predicted label:", predicted_label, mappingId2Label[r[1]])

Predicted label: positive positive
Predicted label: negative negative
Predicted label: negative negative
Predicted label: positive positive
Predicted label: positive negative
Predicted label: positive positive
Predicted label: negative negative
Predicted label: positive positive
Predicted label: positive negative
Predicted label: positive positive
Predicted label: negative negative


In [30]:
from sklearn.metrics import f1_score

# Предобработка текста
test_data['content'] = test_data['text'].apply(lambda x: preprocess(x, stop_words, punctuation_marks, morph))

# Предсказание меток для тестовых данных
y_true = test_data['label']
y_pred = w2v_pipeline.predict(test_data['content'])

# Преобразование предсказанных меток в числовой формат
y_pred = [1 if label == 'positive' else 0 for label in y_pred]

# Оценка F1-меры
f1 = f1_score(y_true, y_pred)
print("F1-мера на тестовых данных:", f1)

F1-мера на тестовых данных: 0.8333333333333333


In [31]:
import pandas as pd

# Замените 'data.csv' на путь к вашему файлу CSV
df = pd.read_csv('train_data.csv')

concatenated_questions_list = []
rel_list = []

# Проходим по каждой строке и конкатенируем значения в столбцах question_1 до question_5
for index, row in df.iterrows():
    concatenated_questions = ' '.join(row[['question_1', 'question_2', 'question_3', 'question_4', 'question_5']])
    concatenated_questions_list.append(concatenated_questions)
    # print(concatenated_questions)

# Проходим по каждой строке и конкатенируем значения в столбцах question_1 до question_5
for index, row in df.iterrows():
    # rel = ''.join(str(row['is_positive']))
    rel = ''.join(str(row['is_relevant']))
    rel_list.append(rel)
print(concatenated_questions_list)
print(rel_list)


['Основы программирования Интерактивный подход к изучению основ C# был очень увлекательным. Нет. Увеличить количество практических заданий для лучшего закрепления материала. Введение в разработку на .NET Framework.', 'Основы программирования Подробное объяснение логических операторов в JavaScript помогло мне лучше понять условные конструкции. Нет. Предоставить ещё больше практических задач после теории. Детальное изучение асинхронного программирования в JavaScript.', 'Основы программирования Введение в программирование на Python было простым и интуитивно понятным, особенно для начинающих. Нет. Улучшить качество видеоматериалов, иногда было трудно разглядеть код. Библиотеки Python для анализа данных и машинного обучения.', 'Основы программирования Понятное изложение основных понятий, хорошая структура материала. Да, циклы for и while были объяснены слишком быстро. Добавить видеоматериалы для самостоятельного изучения. Функциональное программирование.', 'Основы программирования Кларность

In [32]:
from sklearn.metrics import f1_score

# Предобработка текста
# test_data_content_ = concatenated_questions_list.(lambda x: preprocess(x, stop_words, punctuation_marks, morph))
test_data_content_ = list(map(lambda x: preprocess(x, stop_words, punctuation_marks, morph), concatenated_questions_list))
print(test_data_content_)

# Предсказание меток для тестовых данных
y_true = [1 if label == '1' else 0 for label in rel_list]
y_pred = w2v_pipeline.predict(test_data_content_)
print(y_pred)

# # Преобразование предсказанных меток в числовой формат
y_pred = [1 if label == 'positive' else 0 for label in y_pred]

# Оценка F1-меры
f1 = f1_score(y_true, y_pred)
print("F1-мера на тестовых данных:", f1)

['основа программирование интерактивный подход изучение основа c # очень увлекательный увеличить количество практический задание хороший закрепление материал введение разработка .net framework', 'основа программирование подробный объяснение логический оператор javascript помочь хороший понять условный конструкция предоставить ещё большой практический задача теория детальный изучение асинхронный программирование javascript', 'основа программирование введение программирование python простой интуитивно понятный особенно начинающий улучшить качество видеоматериал трудно разглядеть код библиотека python анализ данные машинный обучение', 'основа программирование понятный изложение основный понятие хороший структура материал цикл for while объяснить слишком быстро добавить видеоматериал самостоятельный изучение функциональный программирование', 'основа программирование кларность структура материал основа javascript особенно затронуть тема цикл функция ввести большой пример работа массив объек

### Код is_relevant

In [49]:
import pandas as pd

# Замените 'data.csv' на путь к вашему файлу CSV
df = pd.read_csv('train_data.csv')

concatenated_questions_list = []
rel_list = []

# Проходим по каждой строке и конкатенируем значения в столбцах question_1 до question_5
for index, row in df.iterrows():
    concatenated_questions = ' '.join(row[['question_1', 'question_2', 'question_3', 'question_4', 'question_5']])
    concatenated_questions_list.append(concatenated_questions)
    # print(concatenated_questions)

# Проходим по каждой строке и конкатенируем значения в столбцах question_1 до question_5
for index, row in df.iterrows():
    # rel = ''.join(str(row['is_positive']))
    rel = ''.join(str(row['is_relevant']))
    rel_list.append(rel)
print(concatenated_questions_list)
print(rel_list)

['Основы программирования Интерактивный подход к изучению основ C# был очень увлекательным. Нет. Увеличить количество практических заданий для лучшего закрепления материала. Введение в разработку на .NET Framework.', 'Основы программирования Подробное объяснение логических операторов в JavaScript помогло мне лучше понять условные конструкции. Нет. Предоставить ещё больше практических задач после теории. Детальное изучение асинхронного программирования в JavaScript.', 'Основы программирования Введение в программирование на Python было простым и интуитивно понятным, особенно для начинающих. Нет. Улучшить качество видеоматериалов, иногда было трудно разглядеть код. Библиотеки Python для анализа данных и машинного обучения.', 'Основы программирования Понятное изложение основных понятий, хорошая структура материала. Да, циклы for и while были объяснены слишком быстро. Добавить видеоматериалы для самостоятельного изучения. Функциональное программирование.', 'Основы программирования Кларность

In [50]:
import csv

# mappingId2Label = {0:'negative', 1:'positive'}
mappingLabe2Id = {'negative':0, 'positive':1}

def get_label(text):
    preprocessed_text = preprocess(text, stop_words, punctuation_marks, morph)
    predicted_label = w2v_pipeline.predict([preprocessed_text])[0]
    print(predicted_label)
    return mappingLabe2Id[predicted_label]

# Открываем файл CSV для чтения
with open('sample_submission.csv', 'r') as file:
    # Создаем объект чтения CSV
    csv_reader = csv.reader(file)

    # Читаем заголовок
    header = next(csv_reader)

    # Читаем данные из файла
    data = list(csv_reader)

print(len(data))

# Заменяем каждое значение в колонке is_relevant на 1
for row, text in zip(data, concatenated_questions_list[:len(data)]):
    print(text)
    row[1] = get_label(text)

# Записываем измененные данные обратно в файл CSV
with open('your_submission.csv', 'w', newline='') as file:
    # Создаем объект записи CSV
    csv_writer = csv.writer(file)

    # Записываем заголовок
    csv_writer.writerow(header)

    # Записываем данные в файл
    csv_writer.writerows(data)

53
Основы программирования Интерактивный подход к изучению основ C# был очень увлекательным. Нет. Увеличить количество практических заданий для лучшего закрепления материала. Введение в разработку на .NET Framework.
positive
Основы программирования Подробное объяснение логических операторов в JavaScript помогло мне лучше понять условные конструкции. Нет. Предоставить ещё больше практических задач после теории. Детальное изучение асинхронного программирования в JavaScript.
positive
Основы программирования Введение в программирование на Python было простым и интуитивно понятным, особенно для начинающих. Нет. Улучшить качество видеоматериалов, иногда было трудно разглядеть код. Библиотеки Python для анализа данных и машинного обучения.
positive
Основы программирования Понятное изложение основных понятий, хорошая структура материала. Да, циклы for и while были объяснены слишком быстро. Добавить видеоматериалы для самостоятельного изучения. Функциональное программирование.
positive
Основы п

### Код is_relevant

In [51]:
import pandas as pd

# Замените 'data.csv' на путь к вашему файлу CSV
df = pd.read_csv('train_data.csv')

concatenated_questions_list = []
pos_list = []

# Проходим по каждой строке и конкатенируем значения в столбцах question_1 до question_5
for index, row in df.iterrows():
    concatenated_questions = ' '.join(row[['question_1', 'question_2', 'question_3', 'question_4', 'question_5']])
    concatenated_questions_list.append(concatenated_questions)
    # print(concatenated_questions)

# Проходим по каждой строке и конкатенируем значения в столбцах question_1 до question_5
for index, row in df.iterrows():
    pos = ''.join(str(row['is_positive']))
    # rel = ''.join(str(row['is_relevant']))
    pos_list.append(rel)
print(concatenated_questions_list)
print(pos_list)

['Основы программирования Интерактивный подход к изучению основ C# был очень увлекательным. Нет. Увеличить количество практических заданий для лучшего закрепления материала. Введение в разработку на .NET Framework.', 'Основы программирования Подробное объяснение логических операторов в JavaScript помогло мне лучше понять условные конструкции. Нет. Предоставить ещё больше практических задач после теории. Детальное изучение асинхронного программирования в JavaScript.', 'Основы программирования Введение в программирование на Python было простым и интуитивно понятным, особенно для начинающих. Нет. Улучшить качество видеоматериалов, иногда было трудно разглядеть код. Библиотеки Python для анализа данных и машинного обучения.', 'Основы программирования Понятное изложение основных понятий, хорошая структура материала. Да, циклы for и while были объяснены слишком быстро. Добавить видеоматериалы для самостоятельного изучения. Функциональное программирование.', 'Основы программирования Кларность

In [52]:
import csv

# mappingId2Label = {0:'negative', 1:'positive'}
mappingLabe2Id = {'negative':0, 'positive':1}

def get_label(text):
    preprocessed_text = preprocess(text, stop_words, punctuation_marks, morph)
    predicted_label = w2v_pipeline.predict([preprocessed_text])[0]
    print(predicted_label)
    return mappingLabe2Id[predicted_label]

# Открываем файл CSV для чтения
with open('your_submission.csv', 'r') as file:
    # Создаем объект чтения CSV
    csv_reader = csv.reader(file)

    # Читаем заголовок
    header = next(csv_reader)

    # Читаем данные из файла
    data = list(csv_reader)

print(len(data))

# Заменяем каждое значение в колонке is_relevant на 1
for row, text in zip(data, concatenated_questions_list[:len(data)]):
    print(text)
    row[3] = get_label(text)

# Записываем измененные данные обратно в файл CSV
with open('your_submission.csv', 'w', newline='') as file:
    # Создаем объект записи CSV
    csv_writer = csv.writer(file)

    # Записываем заголовок
    csv_writer.writerow(header)

    # Записываем данные в файл
    csv_writer.writerows(data)

53
Основы программирования Интерактивный подход к изучению основ C# был очень увлекательным. Нет. Увеличить количество практических заданий для лучшего закрепления материала. Введение в разработку на .NET Framework.
positive
Основы программирования Подробное объяснение логических операторов в JavaScript помогло мне лучше понять условные конструкции. Нет. Предоставить ещё больше практических задач после теории. Детальное изучение асинхронного программирования в JavaScript.
positive
Основы программирования Введение в программирование на Python было простым и интуитивно понятным, особенно для начинающих. Нет. Улучшить качество видеоматериалов, иногда было трудно разглядеть код. Библиотеки Python для анализа данных и машинного обучения.
positive
Основы программирования Понятное изложение основных понятий, хорошая структура материала. Да, циклы for и while были объяснены слишком быстро. Добавить видеоматериалы для самостоятельного изучения. Функциональное программирование.
positive
Основы п

# ОБРАБОТКА РЕЗУЛЬТАТА В GPTlike модели

### Код object

In [ ]:
!pip install gradio_client

In [53]:
import pandas as pd

# Замените 'data.csv' на путь к вашему файлу CSV
df = pd.read_csv('train_data.csv')

concatenated_questions_list = []
object_list = []

# Проходим по каждой строке и конкатенируем значения в столбцах question_1 до question_5
for index, row in df.iterrows():
    concatenated_questions = ' '.join(row[['question_2', 'question_3', 'question_4', 'question_5']])
    concatenated_questions_list.append(concatenated_questions)
    # print(concatenated_questions)

# Проходим по каждой строке и конкатенируем значения в столбцах question_1 до question_5
for index, row in df.iterrows():
    obj = ''.join(str(row['object']))
    object_list.append(obj)
print(concatenated_questions_list)
print(object_list)

['Интерактивный подход к изучению основ C# был очень увлекательным. Нет. Увеличить количество практических заданий для лучшего закрепления материала. Введение в разработку на .NET Framework.', 'Подробное объяснение логических операторов в JavaScript помогло мне лучше понять условные конструкции. Нет. Предоставить ещё больше практических задач после теории. Детальное изучение асинхронного программирования в JavaScript.', 'Введение в программирование на Python было простым и интуитивно понятным, особенно для начинающих. Нет. Улучшить качество видеоматериалов, иногда было трудно разглядеть код. Библиотеки Python для анализа данных и машинного обучения.', 'Понятное изложение основных понятий, хорошая структура материала. Да, циклы for и while были объяснены слишком быстро. Добавить видеоматериалы для самостоятельного изучения. Функциональное программирование.', 'Кларность и структура материала по основам JavaScript, особенно затронутые темы циклов и функций. Нет. Ввести больше примеров на 

In [56]:
from gradio_client import Client

client = Client("https://qwen-qwen1-5-72b-chat.hf.space/--replicas/3kh1x/")


def get_label_object(text):

    dox=f"кому направлен отзыв. выбери один варинат наиболее веротяный \
    0 - техническая часть вебинара (Примеры: интерактивный подход к изучению, структура материала, практических задач, просто и понятно, хорошая структура, доступном виде, правильной формулировки вопросов) , \
    1 - программа вебинара (Примеры: тема с подробными примерами, практических задач, подробности реализации, детальный разбор, был очень подробным и полезным, ясное и чёткое, совместилось обучение, примеры на кейсах из жизни, информативной и понятной), \
    2 - преподаватель (Примеры:лектор, лектор имел, материал более живым, обсуждение актуальных тем тендеций, обсуждение применения, освещение, презентация, дискуссия о, лекция о, доклад о, харизма преподователя, преподаватель, он рассказывает, зазнался, умный).\
    {text.replace('.', '').lower()}"
    result = client.predict(
        dox,	# str  in 'Input' Textbox component
        [[ "None", "None" ],],	# Tuple[str | Dict(file: filepath, alt_text: str | None) | None, str | Dict(file: filepath, alt_text: str | None) | None]  in 'Qwen1.5-72B-Chat' Chatbot component
        "None",	# str  in 'parameter_9' Textbox component
        api_name="/model_chat"
    )

    answer = result[1][1][1].lower()
    if "техническую часть" in answer or "технической части" in answer or "техническая часть" in answer or "организация вебинара" in answer or "tech" in answer or "нет определенного варианта" in answer:
        print(0)
        return 0
    elif "преподаватель" in answer or "лектор" in answer or "1 - программе вебинара" in answer or "prepodavatel" in answer:
        print(2)
        return 2
    else:
        print(1)
        return 1
    print("------>", answer, "<------")
    print()

Loaded as API: https://qwen-qwen1-5-72b-chat.hf.space/--replicas/3kh1x/ ✔


In [57]:
# Открываем файл CSV для чтения
with open('your_submission.csv', 'r') as file:
    # Создаем объект чтения CSV
    csv_reader = csv.reader(file)

    # Читаем заголовок
    header = next(csv_reader)

    # Читаем данные из файла
    data = list(csv_reader)

print(len(data))

# Заменяем каждое значение в колонке is_relevant на 1
for row, text in zip(data, concatenated_questions_list[:len(data)]):
    print(text)
    row[2] = get_label_object(text)

# Записываем измененные данные обратно в файл CSV
with open('your_submission.csv', 'w', newline='') as file:
    # Создаем объект записи CSV
    csv_writer = csv.writer(file)

    # Записываем заголовок
    csv_writer.writerow(header)

    # Записываем данные в файл
    csv_writer.writerows(data)

53
Интерактивный подход к изучению основ C# был очень увлекательным. Нет. Увеличить количество практических заданий для лучшего закрепления материала. Введение в разработку на .NET Framework.
0
Подробное объяснение логических операторов в JavaScript помогло мне лучше понять условные конструкции. Нет. Предоставить ещё больше практических задач после теории. Детальное изучение асинхронного программирования в JavaScript.
1
Введение в программирование на Python было простым и интуитивно понятным, особенно для начинающих. Нет. Улучшить качество видеоматериалов, иногда было трудно разглядеть код. Библиотеки Python для анализа данных и машинного обучения.
0
Понятное изложение основных понятий, хорошая структура материала. Да, циклы for и while были объяснены слишком быстро. Добавить видеоматериалы для самостоятельного изучения. Функциональное программирование.
0
Кларность и структура материала по основам JavaScript, особенно затронутые темы циклов и функций. Нет. Ввести больше примеров на раб

### bert классификатор позитивных негативныз

In [58]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text-classification", model="Dpanov2302/sbert-reviews-classification")

config.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.71G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.78M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.71M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [63]:
import csv

# # mappingId2Label = {0:'negative', 1:'positive'}
mappingLabe2Id = {'negative':0, 'positive':1}

# def get_label(text):
#     preprocessed_text = preprocess(text, stop_words, punctuation_marks, morph)
#     predicted_label = w2v_pipeline.predict([preprocessed_text])[0]
#     print(predicted_label)
#     return mappingLabe2Id[predicted_label]

# Открываем файл CSV для чтения
with open('your_submission.csv', 'r') as file:
    # Создаем объект чтения CSV
    csv_reader = csv.reader(file)

    # Читаем заголовок
    header = next(csv_reader)

    # Читаем данные из файла
    data = list(csv_reader)

print(len(data))

# Заменяем каждое значение в колонке is_relevant на 1
for row, text in zip(data, concatenated_questions_list[:len(data)]):
    print(text)
    row[3] = mappingLabe2Id[pipe(text)[0]['label']]
    print(row[3])

# Записываем измененные данные обратно в файл CSV
with open('your_file.csv', 'w', newline='') as file:
    # Создаем объект записи CSV
    csv_writer = csv.writer(file)

    # Записываем заголовок
    csv_writer.writerow(header)

    # Записываем данные в файл
    csv_writer.writerows(data)

53
Интерактивный подход к изучению основ C# был очень увлекательным. Нет. Увеличить количество практических заданий для лучшего закрепления материала. Введение в разработку на .NET Framework.
1
Подробное объяснение логических операторов в JavaScript помогло мне лучше понять условные конструкции. Нет. Предоставить ещё больше практических задач после теории. Детальное изучение асинхронного программирования в JavaScript.
1
Введение в программирование на Python было простым и интуитивно понятным, особенно для начинающих. Нет. Улучшить качество видеоматериалов, иногда было трудно разглядеть код. Библиотеки Python для анализа данных и машинного обучения.
1
Понятное изложение основных понятий, хорошая структура материала. Да, циклы for и while были объяснены слишком быстро. Добавить видеоматериалы для самостоятельного изучения. Функциональное программирование.
1
Кларность и структура материала по основам JavaScript, особенно затронутые темы циклов и функций. Нет. Ввести больше примеров на раб

### Обработка не информативно

In [ ]:
import pandas as pd

# Замените 'data.csv' на путь к вашему файлу CSV
df = pd.read_csv('train_data.csv')

concatenated_questions_list = []
rel_list = []

# Проходим по каждой строке и конкатенируем значения в столбцах question_1 до question_5
for index, row in df.iterrows():
    concatenated_questions = ' '.join(row[['question_1', 'question_2', 'question_3', 'question_4', 'question_5']])
    concatenated_questions_list.append(concatenated_questions)
    # print(concatenated_questions)

# Проходим по каждой строке и конкатенируем значения в столбцах question_1 до question_5
for index, row in df.iterrows():
    rel = ''.join(str(row['is_relevant']))
    rel_list.append(rel)
print(concatenated_questions_list)
print(rel_list)


In [ ]:
from gradio_client import Client

client = Client("https://qwen-qwen1-5-72b-chat.hf.space/--replicas/3kh1x/")

for text, i in zip(concatenated_questions_list, rel_list):

    dox=f"Информативно или не информативно?: {text}"
    result = client.predict(
        dox,	# str  in 'Input' Textbox component
        [[ "None", "None" ],],	# Tuple[str | Dict(file: filepath, alt_text: str | None) | None, str | Dict(file: filepath, alt_text: str | None) | None]  in 'Qwen1.5-72B-Chat' Chatbot component
        "None",	# str  in 'parameter_9' Textbox component
        api_name="/model_chat"
    )

    answer = result[1][1][1].lower()
    print(i)
    if "не информативно" in answer or "не информативным" in answer or "не информативное" in answer or "не является информативным" in answer:
        if "частично информативным" in answer or "выражает личное мнение" in answer or "не является информативным в традиционном смысле" in answer or "содержит как информативные" in answer or "субъективный опыт" in answer:
            print(1)
        else:
            print(0)
    else:
        print(1)
    print("------>", answer, "<------")
    print()

Loaded as API: https://qwen-qwen1-5-72b-chat.hf.space/--replicas/3kh1x/ ✔


KeyboardInterrupt: 